<a href="https://colab.research.google.com/github/cagBRT/computer-vision/blob/master/Yolov3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Need to upload yolov3.weights - it is too big for github

Run all the code cells. <br>
The go do something else for about ~13 minutes


In [ ]:
!git clone -l -s https://github.com/cagBRT/computer-vision.git cloned-repo
%cd cloned-repo
!ls

# **YOLOv3 Architecture**
YOLO is not the most accurate algorithm. RetinaNet, and SSD outperform it in terms of accuracy. <br>

It still, however, was one of the fastest.<br>

But that speed has been traded off for boosts in accuracy in YOLO v3. While the earlier variant ran on 45 FPS on a Titan X, the current version clocks about 30 FPS. This has to do with the increase in complexity of underlying architecture called Darknet.

# **Darknet-53**
YOLO v3 uses a variant of Darknet, which originally had a 53 layer network trained on Imagenet. For the task of detection, 53 more layers are stacked onto it, giving us a **106 layer fully convolutional underlying architecture for YOLO v3**. <br>
This is the reason behind the slowness of YOLO v3 compared to YOLO v2.

*The most salient feature of v3 is that it makes detections at three different scales.*

Which are precisely given by downsampling the dimensions of the input image by 32, 16 and 8 respectively.

The 13 x 13 layer is responsible for detecting large objects, whereas the 52 x 52 layer detects the smaller objects, with the 26 x 26 layer detecting medium objects.

https://towardsdatascience.com/yolo-v3-object-detection-53fb7d3bfe6b

In [ ]:
from IPython.display import Image
Image("/content/YOLO3architecture.png", width=640)

Underneath it uses YOLOv3 model trained on [COCO dataset](https://cocodataset.org/#home) capable of detecting 80 common objects in context.
The common objects are:<br>
person,
bicycle,
car,
motorcycle,
airplane,
bus,
train,
truck,
boat,
traffic light,
fire hydrant,
stop sign,
parking meter,
bench,
bird,
cat,
dog,
horse,
sheep,
cow,
elephant,
bear,
zebra,
giraffe,
backpack,
umbrella,
handbag,
tie,
suitcase,
frisbee,
skis,
snowboard,
sports ball,
kite,
baseball bat,
baseball glove,
skateboard,
surfboard,
tennis racket,
bottle,
wine glass,
cup,
fork,
knife,
spoon,
bowl,
banana,
apple,
sandwich,
orange,
broccoli,
carrot,
hot dog,
pizza,
donut,
cake,
chair,
couch,
potted plant,
bed,
dining table,
toilet,
tv,
laptop,
mouse,
remote,
keyboard,
cell phone,
microwave,
oven,
toaster,
sink,
refrigerator,
book,
clock,
vase,
scissors,
teddy bear,
hair dryer,
toothbrush

In [ ]:
# load the COCO class labels our YOLO model was trained on
LABELS = open("yolo-object-detection/yolo-coco/coco.names").read()

In [ ]:
print(LABELS)

In [ ]:
# import the necessary packages
import numpy as np
import time
import os
import cv2
from google.colab.patches import cv2_imshow

**Get the pretrained YOLOv3 weights**

In [ ]:
! wget "https://pjreddie.com/media/files/yolov3.weights"

**Each label is assigned a color**

In [ ]:
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3),
	dtype="uint8")

**Load the Darknet model**

In [ ]:
# derive the paths to the YOLO weights and model configuration
weightsPath = "/content/cloned-repo/cloned-repo/yolov3.weights"
configPath = "/content/cloned-repo/cloned-repo/yolo-object-detection/yolo-coco/yolov3.cfg"
# load our YOLO object detector trained on COCO dataset (80 classes)
print("[INFO] loading YOLO from disk...")
net = cv2.dnn.readNetFromDarknet(configPath, weightsPath)

**Load an image**

In [ ]:
# load our input image and grab its spatial dimensions
image = cv2.imread("/content/cloned-repo/yolo-object-detection/images/baggage_claim.jpg")
(H, W) = image.shape[:2]

In [ ]:
# determine only the *output* layer names that we need from YOLO
ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
# construct a blob from the input image and then perform a forward
# pass of the YOLO object detector, giving us our bounding boxes and
# associated probabilities
blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (416, 416),
	swapRB=True, crop=False)
net.setInput(blob)
start = time.time()
layerOutputs = net.forward(ln)
end = time.time()
# show timing information on YOLO
print("[INFO] YOLO took {:.6f} seconds".format(end - start))

**Set up the lists for the bounding boxes, confidences, and classIDs**

In [ ]:
# initialize our lists of detected bounding boxes, confidences, and
# class IDs, respectively
boxes = []
confidences = []
classIDs = []

In [ ]:
# loop over each of the layer outputs
for output in layerOutputs:
	# loop over each of the detections
	for detection in output:
		# extract the class ID and confidence (i.e., probability) of
		# the current object detection
		scores = detection[5:]
		classID = np.argmax(scores)
		confidence = scores[classID]
		# filter out weak predictions by ensuring the detected
		# probability is greater than the minimum probability
		if confidence > .50:
			# scale the bounding box coordinates back relative to the
			# size of the image, keeping in mind that YOLO actually
			# returns the center (x, y)-coordinates of the bounding
			# box followed by the boxes' width and height
			box = detection[0:4] * np.array([W, H, W, H])
			(centerX, centerY, width, height) = box.astype("int")
			# use the center (x, y)-coordinates to derive the top and
			# and left corner of the bounding box
			x = int(centerX - (width / 2))
			y = int(centerY - (height / 2))
			# update our list of bounding box coordinates, confidences,
			# and class IDs
			boxes.append([x, y, int(width), int(height)])
			confidences.append(float(confidence))
			classIDs.append(classID)

In [ ]:
print(boxes)
print(confidences)
print(classIDs)

In [ ]:
# apply non-maxima suppression to suppress weak, overlapping bounding
# boxes
idxs = cv2.dnn.NMSBoxes(boxes, confidences, .50,0.3)

In [ ]:
# ensure at least one detection exists
if len(idxs) > 0:
	# loop over the indexes we are keeping
	for i in idxs.flatten():
		# extract the bounding box coordinates
		(x, y) = (boxes[i][0], boxes[i][1])
		(w, h) = (boxes[i][2], boxes[i][3])
		# draw a bounding box rectangle and label on the image
		color = [int(c) for c in COLORS[classIDs[i]]]
		cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
		text = "{}: {:.4f}".format(LABELS[classIDs[i]], confidences[i])
		cv2.putText(image, text, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX,
			0.5, color, 2)
# show the output image
cv2_imshow(image)

In [ ]:
image = cv2.imread("/content/cloned-repo/yolo-object-detection/images/baggage_claim.jpg")
(H, W) = image.shape[:2]

In [ ]:
!pip3 install opencv-python tensorflow
!pip3 install drawbox
!pip install --upgrade cvlib

In [ ]:
import cv2
import matplotlib.pyplot as plt
import cvlib as cv
from cvlib.object_detection import draw_bbox
from google.colab.patches import cv2_imshow

In [ ]:
bbox, label, conf = cv.detect_common_objects(image)
output_image2 = draw_bbox(image, bbox, label, conf)
plt.imshow(output_image2)
plt.show()

In [ ]:
import pandas as pd
ds1 = pd.Series(bbox)
ds2 = pd.Series(conf)
ds3 = pd.Series(label)
dframe={'boxes':ds1,'confidences': ds2,'classIDs': ds3}
df=pd.DataFrame(dframe)
df